In [1]:
import os
from pathlib import Path
import pandas as pd


In [2]:
# ============== CẤU HÌNH ĐƯỜNG DẪN ==============
# Thư mục DAILY đã duyệt: mỗi năm là 1 thư mục, trong đó chứa các CSV theo bang
SRC_BASE   = r"C:\Downloads\Corn_data_3_region\corn_weather_3region"
# File weekly TỔNG (một file duy nhất)
OUT_FILE   = r"C:\Downloads\Corn_data_merged_3_region\weather_weekly_all_years_3region.csv"

In [3]:
# ============== MAPPING TÊN BANG & VÙNG ==============
STATE_NAME = {
    "CO":"COLORADO","IA":"IOWA","IL":"ILLINOIS","IN":"INDIANA","KS":"KANSAS","KY":"KENTUCKY",
    "MI":"MICHIGAN","MN":"MINNESOTA","MO":"MISSOURI","NC":"NORTH CAROLINA","ND":"NORTH DAKOTA",
    "NE":"NEBRASKA","OH":"OHIO","PA":"PENNSYLVANIA","SD":"SOUTH DAKOTA","TN":"TENNESSEE",
    "TX":"TEXAS","WI":"WISCONSIN"
}
REGION_MAP = {
    # Temperate Humid (9)
    "IA":"Temperate Humid","IL":"Temperate Humid","IN":"Temperate Humid","MO":"Temperate Humid",
    "OH":"Temperate Humid","MI":"Temperate Humid","MN":"Temperate Humid","WI":"Temperate Humid","PA":"Temperate Humid",
    # Semi-Arid Continental (6)
    "ND":"Semi-Arid Continental","SD":"Semi-Arid Continental","NE":"Semi-Arid Continental",
    "KS":"Semi-Arid Continental","CO":"Semi-Arid Continental","TX":"Semi-Arid Continental",
    # Subtropical Humid (3)
    "KY":"Subtropical Humid","TN":"Subtropical Humid","NC":"Subtropical Humid"
}

In [4]:
# ============== CHUYỂN 1 FILE DAILY → WEEKLY ==============
def daily_to_weekly(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df.columns = [c.strip().upper() for c in df.columns]

    # bỏ WTxx
    df = df[[c for c in df.columns if not c.startswith("WT")]].copy()

    # kiểm tra cột bắt buộc
    if "STATE" not in df.columns or "DATE" not in df.columns:
        raise ValueError(f"{csv_path.name}: thiếu STATE hoặc DATE")

    # chuẩn DATE
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["DATE"]).set_index("DATE").sort_index()

    # ép numeric cho các cột khí hậu
    for col in ["PRCP","RH","TMAX","TMIN","TAVG"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # resample tuần kết thúc CHỦ NHẬT (bao gồm Chủ nhật)
    res = df.resample("W-SUN", label="right", closed="right")

    # lấy key nhận diện & meta
    state_alpha = res["STATE"].first().rename("state_alpha")
    state_name  = state_alpha.map(STATE_NAME).rename("state_name")
    region      = state_alpha.map(REGION_MAP).rename("region")

    # tính chỉ số tuần
    out = pd.DataFrame({
        "state_alpha": state_alpha,
        "state_name":  state_name,
        "region":      region,
        "tmax":     res["TMAX"].mean(),
        "tmin":     res["TMIN"].mean(),
        "tavg":     res["TAVG"].mean(),
        "prcp_sum": res["PRCP"].sum(min_count=1),   # tổng mưa tuần
        "prcp_mean":res["PRCP"].mean(),             # trung bình mưa ngày trong tuần
        "rh":       res["RH"].mean(),
    }).reset_index().rename(columns={"DATE":"week_ending"})

    # thêm year (từ week_ending)
    out["year"] = out["week_ending"].dt.year.astype(int)

    # thứ tự cột chuẩn
    out = out[["state_alpha","state_name","year","region","week_ending",
               "tmax","tmin","tavg","prcp_sum","prcp_mean","rh"]]
    return out


In [5]:
# ============== QUÉT TẤT CẢ NĂM & GỘP ==============
def build_all_years(src_base: str) -> pd.DataFrame:
    frames = []
    for year_dir in sorted(Path(src_base).iterdir()):
        if not year_dir.is_dir():
            continue
        for csv_path in sorted(year_dir.glob("*.csv")):
            try:
                wk = daily_to_weekly(csv_path)
                
                wk = wk[wk["year"] == int(year_dir.name)]
                frames.append(wk)
            except Exception as e:
                print(f"[WARN] {csv_path.name}: {e}")
    if not frames:
        raise SystemExit("Không thu được dữ liệu nào.")
    weekly_all = (pd.concat(frames, ignore_index=True)
                    .sort_values(["state_alpha","year","week_ending"])
                    .reset_index(drop=True))
    return weekly_all


In [6]:
if __name__ == "__main__":
    weekly_all = build_all_years(SRC_BASE)
    out_path = Path(OUT_FILE)
    out_dir  = out_path.parent

    # nếu chưa có file tổng thì tạo mới, nếu đã có thì append thêm
    mode, header = ("w", True) if not out_path.exists() else ("a", False)
    weekly_all.to_csv(out_path, mode=mode, header=header, index=False, encoding="utf-8-sig")


    print(f" Đã tạo file weekly tổng: {out_path}")
    print(f"   Số dòng: {len(weekly_all):,} | Số cột: {len(weekly_all.columns)}")
    print("   Cột:", weekly_all.columns.tolist())



 Đã tạo file weekly tổng: C:\Downloads\Corn_data_merged_3_region\weather_weekly_all_years_3region.csv
   Số dòng: 18,792 | Số cột: 11
   Cột: ['state_alpha', 'state_name', 'year', 'region', 'week_ending', 'tmax', 'tmin', 'tavg', 'prcp_sum', 'prcp_mean', 'rh']
